In [ ]:
import tensorflow as tf
import numpy as np
import importlib.util
import matplotlib.pyplot as plt
%matplotlib inline

# Check the normalisation by feeding with normal random values

In [ ]:
def load_module(path):
    spec = importlib.util.spec_from_file_location("module.name", path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def load_arch(arch_path):
    arch = load_module(arch_path)
    nn = arch.CNN()

    g = tf.Graph()
    with g.as_default():
        nn.create_architecture()
    return g, nn

In [ ]:
graph, nn = load_arch("arch_invariant_c.py")

In [ ]:
sess = tf.Session(graph=graph)
sess.run(tf.variables_initializer(graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)))

In [ ]:
tensors = [o.inputs[0] for o in graph.get_operations() if not 'gradients' in o.name 
           and ('Conv2D' in o.name or 'MatMul' in o.name or 'MaxPool' in o.name or 'Sigmoid' in o.name)]

In [ ]:
x = np.random.normal(size=(10, 424, 424, 3))
y = sess.run(tensors, feed_dict={nn.tfx: x})

In [ ]:
[(np.prod(t.shape[1:]), t.shape, np.mean(t), np.std(t)) for t in y]

## Normalisation propagation for pool max 2x2

In [ ]:
x = np.random.normal(size=(1000000, 4))
x = np.amax(x, 1)
m, s = (np.mean(x), np.std(x))
a, b = 1/s, m/s
(m,s),(a,b)

In [ ]:
x = np.random.normal(size=(1000000, 4))
x = a * x - b
x = np.amax(x, 1)
np.mean(x), np.std(x)